In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
from tensorflow.keras import layers, models, utils
import time
import numpy as np
data_dir = './data/cinic-10_image_classification_challenge-dataset/train/'

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
def load_and_preprocess_image(image_path, label):
    # Load image
    image = tf.io.read_file(image_path)
    # Decode PNG image to tensor
    image = tf.image.decode_png(image, channels=3)  # Adjust channels according to your images
    # Normalize pixel values to range [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image, label


def preprocess_data():

    class_names = os.listdir(data_dir)

    image_paths = []
    labels = []
    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            image_paths.append(image_path)
            labels.append(class_names.index(class_name))

    # Create TensorFlow Dataset from the loaded data
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(load_and_preprocess_image)

    dataset = dataset.shuffle(buffer_size=10000)

# Split the dataset into train, validation, and test sets
    train_size = int(0.8 * len(dataset))
    test_size = int(0.2 * len(dataset))

    train_dataset = dataset.take(train_size)
    test_dataset = dataset.skip(train_size).take(test_size)
    train_dataset = train_dataset.shuffle(buffer_size=len(image_paths)).batch(32)
    test_dataset = test_dataset.shuffle(buffer_size=len(image_paths)).batch(32)
    return train_dataset, test_dataset

In [ ]:
train_dataset, test_dataset = preprocess_data()

In [3]:
train_dataset = utils.image_dataset_from_directory(
    "./data/cinic-10_image_classification_challenge-dataset/train",
    image_size=(32, 32),
    label_mode="categorical",
    seed=123,
)

Found 90000 files belonging to 10 classes.


In [ ]:
train_dataset

## AlexNet

In [ ]:
model = models.Sequential()
model.add(layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=(32, 32, 3)))
model.add(layers.Conv2D(96, 11, strides=4, padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))
model.add(layers.Conv2D(256, 5, strides=4, padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))
model.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(256, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(
    train_dataset,
    epochs=5,
    # validation_data=validation_dataset,
)

In [ ]:

history.history['accuracy']

In [ ]:
ann = models.Sequential(
    [
        layers.Rescaling(1.0 / 255, input_shape=(32, 32, 3)),
        layers.Flatten(input_shape=(32, 32, 3)),
        layers.Dense(3000, activation="relu"),
        layers.Dense(1000, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)

ann.compile(optimizer="SGD", loss="categorical_crossentropy", metrics=["accuracy"])

ann.fit(
    train_dataset,
    epochs=5,
    # validation_data=validation_dataset,
)

## Resnet

In [4]:
from tensorflow.keras import datasets, layers, models, losses, Model

In [5]:
base_model152 = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model152.layers:
  layer.trainable = False


base_model101 = tf.keras.applications.ResNet101(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model101.layers:
  layer.trainable = False


base_model50 = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model50.layers:
  layer.trainable = False

In [ ]:
base_model152 = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model152.layers:
  layer.trainable = False

x = layers.Flatten()(base_model152.output)
x = layers.Dense(1000, activation='relu')(x)
predictions = layers.Dense(10, activation = 'softmax')(x)

head_model = Model(inputs = base_model152.input, outputs = predictions)
head_model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

history = head_model.fit(train_dataset, epochs=5)

In [6]:
with open('results.txt', 'a') as file:
    for model, model_nr in zip([base_model50, base_model101, base_model152], ['50', '101', '152']):
        for optimize_algo in ['SGD', 'Adam', 'Adagrad']:
            for activation_func in ['relu', 'linear', 'softmax']:
                for regularization_algo, regularization_algo_name in zip([tf.keras.regularizers.L1, tf.keras.regularizers.L2], ['l1', 'l2']):
                    for l in [0.1, 0.3, 0.5]:

                        x = layers.Flatten()(model.output)
                        x = layers.Dense(1000, activation=activation_func, kernel_regularizer=regularization_algo(l))(x)
                        predictions = layers.Dense(10, activation = 'softmax')(x)

                        head_model = Model(inputs = model.input, outputs = predictions)
                        head_model.compile(optimizer=optimize_algo, loss="categorical_crossentropy", metrics=["accuracy"])

                        history = head_model.fit(train_dataset, epochs=3)
                        print(model_nr, optimize_algo, activation_func, regularization_algo_name, l, max(history.history['accuracy']))
                        file.write(model_nr + ' ' + optimize_algo + ' ' + activation_func + ' ' + regularization_algo_name + ' ' + str(l) + ' ' + str(max(history.history['accuracy'])) + '\n')

Epoch 1/3
2813/2813 [==============================] - 107s 36ms/step - loss: 129.5083 - accuracy: 0.1716
Epoch 2/3
2813/2813 [==============================] - 101s 36ms/step - loss: 104.9954 - accuracy: 0.2035
Epoch 3/3
2813/2813 [==============================] - 102s 36ms/step - loss: 104.8715 - accuracy: 0.2162
50 SGD relu l1 0.1 0.21622222661972046
Epoch 1/3
2813/2813 [==============================] - 105s 37ms/step - loss: 948.7927 - accuracy: 0.1294
Epoch 2/3
2813/2813 [==============================] - 102s 36ms/step - loss: 924.3524 - accuracy: 0.1364
Epoch 3/3
2813/2813 [==============================] - 102s 36ms/step - loss: 924.2646 - accuracy: 0.1406
50 SGD relu l1 0.3 0.14063332974910736
Epoch 1/3
2813/2813 [==============================] - 105s 36ms/step - loss: 2586.9675 - accuracy: 0.1303
Epoch 2/3
2813/2813 [==============================] - 102s 36ms/step - loss: 2562.7554 - accuracy: 0.1336
Epoch 3/3
2813/2813 [==============================] - 101s 36ms/step - 

KeyboardInterrupt: 

In [9]:
import pandas as pd

In [12]:
data = pd.read_csv('results.txt', sep=' ', header=None)

In [16]:
data.sort_values(5, ascending=False)

,0,1,2,3,4,5
39,50,Adagrad,relu,l2,0.1,0.516589
93,101,Adagrad,relu,l2,0.1,0.509022
45,50,Adagrad,linear,l2,0.1,0.506867
99,101,Adagrad,linear,l2,0.1,0.498456
40,50,Adagrad,relu,l2,0.3,0.493600
...,...,...,...,...,...,...
13,50,SGD,softmax,l1,0.3,0.099689
67,101,SGD,softmax,l1,0.3,0.099689
48,50,Adagrad,softmax,l1,0.1,0.099533
50,50,Adagrad,softmax,l1,0.5,0.097022


In [22]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Num GPUs Available:  1
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [23]:
tf.debugging.set_log_device_placement(True)
tf.ones([])
# [...] op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
with tf.device("CPU"):
 tf.ones([])
# [...] op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
tf.debugging.set_log_device_placement(False)